In [98]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl

import matplotlib.pyplot as plt
import ydata_profiling as pp
import seaborn as sns
import datetime as dt

from pandas.tseries.offsets import DateOffset
from sklearn.model_selection import train_test_split

In [138]:
PATH = '../data/'
clients = pd.read_csv(PATH + '/clients.csv')
report_dates = pd.read_csv(PATH + 'report_dates.csv', parse_dates=['report_dt'])


transactions = pd.read_csv(PATH + 'transactions.csv', parse_dates=['transaction_dttm'])
train_data = pd.read_csv(PATH + 'train.csv')

In [77]:
# Добваляем сколько дней прошло с даты последней транзакции
clients = clients.merge(report_dates, on="report", how="left")
clients['cencoring'] = clients['report_dt'] + DateOffset(months=-3)
transactions = transactions.merge(clients[['user_id', 'cencoring']])
transactions['recency'] = (transactions['cencoring']-transactions['transaction_dttm']).dt.days
# Разбиваем на позитивные и негативные
transactions['positive'] = transactions['transaction_amt'].apply(lambda x: x if x>0 else  0)
transactions['negative'] = transactions['transaction_amt'].apply(lambda x: x if x<0 else  0)



In [99]:
transactions = pl.DataFrame(transactions)
rub_df = transactions.filter(transactions['currency_rk']==1)
not_rub_df = transactions.filter(transactions['currency_rk']!=1)
transactions



user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,cencoring,recency,positive,negative
i64,i64,i64,f64,datetime[ns],datetime[ns],i64,f64,f64
3,3,1,-183.883957,2022-01-28 12:05:33,2022-05-31 03:00:00,122,0.0,-183.883957
3,3,1,-3206.437012,2022-01-28 12:52:30,2022-05-31 03:00:00,122,0.0,-3206.437012
3,16,1,-153866.890625,2022-02-16 14:45:56,2022-05-31 03:00:00,103,0.0,-153866.890625
3,56,1,-15144.601562,2022-03-09 19:58:29,2022-05-31 03:00:00,82,0.0,-15144.601562
3,0,1,5297.908691,2022-03-12 18:11:31,2022-05-31 03:00:00,79,5297.908691,0.0
3,0,1,534.363098,2022-04-11 20:48:26,2022-05-31 03:00:00,49,534.363098,0.0
3,0,1,4549.455078,2022-04-11 20:53:35,2022-05-31 03:00:00,49,4549.455078,0.0
3,0,1,37991.929688,2022-04-29 18:42:58,2022-05-31 03:00:00,31,37991.929688,0.0
3,0,1,28335.613281,2022-05-05 20:32:38,2022-05-31 03:00:00,25,28335.613281,0.0


In [105]:
grouped_rub = rub_df.group_by('user_id').agg(
    pl.col('recency').min().alias('last_recency'),
    pl.col('recency').max().alias('first_recency'),
    pl.col('recency').diff().mean().abs().alias('diff_recency'),

    pl.col('transaction_amt').count().alias('count'),
    pl.col('positive').sum().alias('sum_positive'),
    pl.col('negative').max().alias('sum_negative'),
    pl.col('mcc_code').n_unique().alias('n_unique_mcc')
)

grouped_not_rub = not_rub_df.group_by('user_id').agg(
    pl.col('recency').min().alias('last_recency'),
    pl.col('recency').max().alias('first_recency'),
    pl.col('recency').diff().mean().abs().alias('diff_recency'),

    pl.col('transaction_amt').count().alias('count'),
    pl.col('positive').sum().alias('sum_positive'),
    pl.col('negative').max().alias('sum_negative'),
    pl.col('mcc_code').n_unique().alias('n_unique_mcc')
)

In [114]:
data= grouped_rub.join(grouped_not_rub, how='outer_coalesce', on='user_id', suffix='_not_rub')
data = data.fill_null(0)

In [117]:
data

user_id,last_recency,first_recency,diff_recency,count,sum_positive,sum_negative,n_unique_mcc,last_recency_not_rub,first_recency_not_rub,diff_recency_not_rub,count_not_rub,sum_positive_not_rub,sum_negative_not_rub,n_unique_mcc_not_rub
i64,i64,i64,f64,u32,f64,f64,u32,i64,i64,f64,u32,f64,f64,u32
176964,11,12,0.5,3,6568.759033,0.0,2,38,190,4.222222,37,38510.330566,0.0,9
454677,34,187,3.1875,49,7989.890625,0.0,7,0,0,0.0,0,0.0,0.0,0
489913,15,191,0.709677,249,500571.989014,0.0,24,0,0,0.0,0,0.0,0.0,0
56697,10,188,1.663551,108,163100.373589,0.0,14,0,0,0.0,0,0.0,0.0,0
198335,28,191,3.075472,54,199129.286163,0.0,8,0,0,0.0,0,0.0,0.0,0
501828,16,61,2.647059,18,6558.675888,0.0,7,63,187,4.96,26,7606.642151,0.0,7
529335,9,189,1.8,101,5279.828857,0.0,9,0,0,0.0,0,0.0,0.0,0
170194,9,191,0.223039,817,20064.210876,0.0,49,0,0,0.0,0,0.0,0.0,0
168756,10,190,1.005587,180,30943.087036,0.0,32,134,185,8.5,7,5872.403076,0.0,2


In [120]:
# data = data.to_pandas()
quantiles = data.quantile([0.05,0.25,0.5,0.75,0.95])
quantiles

,user_id,last_recency,first_recency,diff_recency,count,sum_positive,sum_negative,n_unique_mcc,last_recency_not_rub,first_recency_not_rub,diff_recency_not_rub,count_not_rub,sum_positive_not_rub,sum_negative_not_rub,n_unique_mcc_not_rub
0.05,27510.85,9.0,119.0,0.406250,13.0,0.000000,-94.476940,2.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
0.25,136955.00,10.0,181.0,0.927083,32.0,0.000000,-6.641456,7.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
0.50,277938.00,12.0,189.0,2.011111,79.0,9576.326263,0.000000,13.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
0.75,414695.25,17.0,191.0,4.666667,182.0,71192.131369,0.000000,21.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
0.95,524111.75,39.0,193.0,11.454545,420.0,383086.319598,0.000000,36.0,119.0,187.0,4.692308,16.0,22662.382912,0.0,4.0


In [ ]:

data=transactions.groupby('user_id').agg({'recency': 'min', # Recency
                                                'transaction_dttm': 'count', # Frequency
                                                'transaction_amt': 'sum'})    # Monetary

data.rename(columns={'transaction_dttm': 'frequency',
                         'transaction_amt': 'monetary_value'}, inplace=True)


quantiles = data.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [121]:
# def R_Class(x,p,d):
#     if x <= d[p][0.25]:
#         return 4
#     elif x <= d[p][0.50]:
#         return 3
#     elif x <= d[p][0.75]: 
#         return 2
#     else:
#         return 1
    

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FM_Class(x,p,d):
    if x == 0: 
        return 0
    elif x <= d[p][0.05]:
        return 1
    elif x <= d[p][0.25]:
        return 2
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 3
    elif x <= d[p][0.95]:
        return 4
    else:
        return 5
    


In [129]:
data.columns.difference(['user_id'])

Index(['count', 'count_not_rub', 'diff_recency', 'diff_recency_not_rub',
       'first_recency', 'first_recency_not_rub', 'last_recency',
       'last_recency_not_rub', 'n_unique_mcc', 'n_unique_mcc_not_rub',
       'sum_negative', 'sum_negative_not_rub', 'sum_positive',
       'sum_positive_not_rub'],
      dtype='object')

In [131]:
for col in data.columns.difference(['user_id']):
    data[col] = data[col].apply(FM_Class, args=(col,quantiles,))

In [132]:
data

,user_id,last_recency,first_recency,diff_recency,count,sum_positive,sum_negative,n_unique_mcc,last_recency_not_rub,first_recency_not_rub,diff_recency_not_rub,count_not_rub,sum_positive_not_rub,sum_negative_not_rub,n_unique_mcc_not_rub
0,176964,3,1,2,1,3,0,1,4,5,4,5,5,0,5
1,454677,4,3,3,3,3,0,2,0,0,0,0,0,0,0
2,489913,3,3,2,4,5,0,4,0,0,0,0,0,0,0
3,56697,2,3,3,3,4,0,3,0,0,0,0,0,0,0
4,198335,4,3,3,3,4,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,502946,0,0,0,0,0,0,0,4,4,5,4,4,0,4
95996,497599,0,0,0,0,0,0,0,4,5,4,5,5,0,5
95997,497634,0,0,0,0,0,0,0,4,5,5,5,4,0,5
95998,133772,0,0,0,0,0,0,0,4,4,5,4,4,0,5


In [139]:
clients = clients.merge(data, how='left', on='user_id')

In [142]:
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':1,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)


clients.fillna(0, inplace=True)

/tmp/ipykernel_4876/2680233067.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':1,


In [140]:
# data['R_Quartile'] = data['recency'].apply(R_Class, args=('recency',quantiles,))
# data['F_Quartile'] = data['frequency'].apply(FM_Class, args=('frequency',quantiles,))
# data['M_Quartile'] = data['monetary_value'].apply(FM_Class, args=('monetary_value',quantiles,))
# data['RFMClass'] = data.R_Quartile.map(str) \
#                             + data.F_Quartile.map(str) \
#                             + data.M_Quartile.map(str)

In [143]:
clients

,user_id,report,employee_count_nm,bankemplstatus,customer_age,last_recency,first_recency,diff_recency,count,sum_positive,sum_negative,n_unique_mcc,last_recency_not_rub,first_recency_not_rub,diff_recency_not_rub,count_not_rub,sum_positive_not_rub,sum_negative_not_rub,n_unique_mcc_not_rub
0,3,2,4.0,0,3,3,2,4,1,4,0,2,0,0,0,0,0,0,0
1,9,1,6.0,0,3,2,3,3,3,0,2,4,0,0,0,0,0,0,0
2,13,6,5.0,0,2,4,3,4,2,4,0,2,5,4,0,4,4,0,4
3,37,5,6.0,0,2,3,3,2,4,3,0,4,0,0,0,0,0,0,0
4,41,1,4.0,0,2,3,2,4,2,0,1,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,12,0.0,0,2,5,2,3,3,0,2,3,0,0,0,0,0,0,0
95996,562205,12,0.0,0,1,2,3,3,3,3,0,3,0,0,0,0,0,0,0
95997,562312,12,0.0,0,0,4,3,3,3,0,2,2,0,0,0,0,0,0,0
95998,562721,12,0.0,0,2,3,3,3,3,3,0,3,0,0,0,0,0,0,0


In [53]:
clients = clients.join(data[['R_Quartile','F_Quartile','M_Quartile']], how='left', on='user_id')

In [65]:
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':4,'БОЛЕЕ 1001':6,'ОТ 501 ДО 1000':5,'ДО 10':1,
                                      'ОТ 11 ДО 50':2,'ОТ 51 ДО 100':3,'БОЛЕЕ 500':5,'ОТ 11 ДО 30':2,
                                      'ОТ 31 ДО 50':2}}, inplace=True)


clients.fillna(0, inplace=True)
# clients.drop(columns= ['report_dt', 'cencoring'], inplace=True)

In [144]:
data = clients.merge(train_data, on="user_id", how="left")

In [145]:
data_train = data[data['target'].notna()]
data_test = data[data['target'].isna()]

In [146]:
X_train, X_valid = train_test_split(data_train, test_size=0.25, random_state=1)

In [147]:
data_train

,user_id,report,employee_count_nm,bankemplstatus,customer_age,last_recency,first_recency,diff_recency,count,sum_positive,...,n_unique_mcc,last_recency_not_rub,first_recency_not_rub,diff_recency_not_rub,count_not_rub,sum_positive_not_rub,sum_negative_not_rub,n_unique_mcc_not_rub,target,time
0,3,2,4.0,0,3,3,2,4,1,4,...,2,0,0,0,0,0,0,0,0.0,77.0
2,13,6,5.0,0,2,4,3,4,2,4,...,2,5,4,0,4,4,0,4,0.0,86.0
3,37,5,6.0,0,2,3,3,2,4,3,...,4,0,0,0,0,0,0,0,0.0,89.0
4,41,1,4.0,0,2,3,2,4,2,0,...,2,0,0,0,0,0,0,0,0.0,57.0
5,42,12,1.0,0,3,2,3,3,3,4,...,3,0,0,0,0,0,0,0,0.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95992,561824,12,0.0,0,0,2,3,3,3,3,...,4,0,0,0,0,0,0,0,0.0,91.0
95995,562043,12,0.0,0,2,5,2,3,3,0,...,3,0,0,0,0,0,0,0,0.0,75.0
95997,562312,12,0.0,0,0,4,3,3,3,0,...,2,0,0,0,0,0,0,0,0.0,91.0
95998,562721,12,0.0,0,2,3,3,3,3,3,...,3,0,0,0,0,0,0,0,0.0,29.0


In [70]:

from lifelines import CoxPHFitter

In [148]:
cph = CoxPHFitter()
cph.fit(X_train.drop('user_id', axis=1), 'time', 'target')

<lifelines.CoxPHFitter: fitted with 48000 total observations, 43896 right-censored observations>

In [150]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 48000 total observations, 43896 right-censored observations>
             duration col = 'time'
                event col = 'target'
      baseline estimation = breslow
   number of observations = 48000
number of events observed = 4104
   partial log-likelihood = -42095.31
         time fit was run = 2024-03-27 10:57:36 UTC

---
                        coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                               
report                 -0.04      0.96      0.00           -0.05           -0.03                0.95                0.97
employee_count_nm      -0.19      0.83      0.01           -0.21           -0.18                0.81                0.84
bankemplstatus        -13.61      0.00    300.71         -603.00          575.77                0.00           1.13e+250
customer_age           -0.27      0.76      0.01           -0.30           -0.25                0.74                0.78
last_recency            0.30      1.35      0.01            0.27            0.33                1.31                1.39
first_recency          -0.31      0.73      0.03           -0.37           -0.26                0.69                0.77
diff_recency           -0.16      0.85      0.02           -0.21           -0.11                0.81                0.89
count                   0.14      1.15      0.03            0.08            0.21                1.08                1.23
sum_positive           -0.06      0.94      0.02           -0.10           -0.02                0.90                0.98
sum_negative           -0.10      0.91      0.04           -0.17           -0.02                0.84                0.98
n_unique_mcc           -0.33      0.72      0.03           -0.38           -0.28                0.69                0.76
last_recency_not_rub    0.25      1.29      0.07            0.12            0.38                1.13                1.47
first_recency_not_rub   0.14      1.16      0.09           -0.02            0.31                0.98                1.37
diff_recency_not_rub   -0.04      0.96      0.03           -0.10            0.03                0.90                1.03
count_not_rub           0.42      1.52      0.12            0.20            0.65                1.22                1.91
sum_positive_not_rub   -0.65      0.52      0.09           -0.82           -0.48                0.44                0.62
sum_negative_not_rub   -2.64      0.07      0.39           -3.41           -1.87                0.03                0.15
n_unique_mcc_not_rub   -0.16      0.86      0.11           -0.37            0.06                0.69                1.06

                       cmp to      z      p  -log2(p)
covariate                                            
report                   0.00  -7.67 <0.005     45.69
employee_count_nm        0.00 -24.72 <0.005    445.88
bankemplstatus           0.00  -0.05   0.96      0.05
customer_age             0.00 -19.99 <0.005    292.89
last_recency             0.00  20.31 <0.005    302.32
first_recency            0.00 -11.62 <0.005    101.27
diff_recency             0.00  -6.52 <0.005     33.76
count                    0.00   4.22 <0.005     15.34
sum_positive             0.00  -2.72   0.01      7.24
sum_negative             0.00  -2.51   0.01      6.36
n_unique_mcc             0.00 -12.57 <0.005    118.03
last_recency_not_rub     0.00   3.79 <0.005     12.70
first_recency_not_rub    0.00   1.68   0.09      3.42
diff_recency_not_rub     0.00  -1.06   0.29      1.79
count_not_rub            0.00   3.66 <0.005     11.96
sum_positive_not_rub     0.00  -7.53 <0.005     44.21
sum_negative_not_rub     0.00  -6.70 <0.005     35.44
n_unique_mcc_not_rub     0.00  -1.42   0.15      2.69
---
Concordance = 0.72
Partial AIC = 84226.61
log-likelihood ratio test = 2752.02 on 18 df
-log2(p) of ll-ratio test = inf

In [152]:
X_train, X_valid,y_train, y_valid = train_test_split(data_train.drop(['user_id','target','time'], axis=1),data_train['target'], test_size=0.25, random_state=1)

In [154]:
from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,

    eval_metric='AUC',
    # cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(Pool(X_train, y_train),
          eval_set=Pool(X_valid, y_valid),
           verbose=100)

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	test: 0.6265268	best: 0.6265268 (0)	total: 67.3ms	remaining: 1m 34s
100:	test: 0.7218995	best: 0.7218995 (100)	total: 612ms	remaining: 7.87s
200:	test: 0.7285414	best: 0.7285489 (199)	total: 1.14s	remaining: 6.83s
300:	test: 0.7317019	best: 0.7317634 (297)	total: 1.66s	remaining: 6.06s
400:	test: 0.7329213	best: 0.7330035 (363)	total: 2.18s	remaining: 5.42s
500:	test: 0.7336011	best: 0.7336011 (500)	total: 2.69s	remaining: 4.82s
600:	test: 0.7333283	best: 0.7337542 (529)	total: 3.2s	remaining: 4.25s
700:	test: 0.7331042	best: 0.7337542 (529)	total: 3.71s	remaining: 3.7s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7337541876
bestIteration = 529

Shrink model to first 530 iterations.


In [155]:

df_imp = pd.DataFrame({
    'name': X_train.columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

In [156]:
df_imp

,name,imp
1,employee_count_nm,20.198884
3,customer_age,14.305485
4,last_recency,10.980776
10,n_unique_mcc,10.585298
0,report,7.552389
6,diff_recency,6.551410
5,first_recency,6.088128
8,sum_positive,5.077335
7,count,4.712882
11,last_recency_not_rub,2.387724
